In [1]:
!python --version

Python 3.11.13


In [2]:
!where python

C:\Users\yvnyl\anaconda3\envs\env-ml-zoomcamp-2025\python.exe
C:\Users\yvnyl\anaconda3\python.exe
C:\Python313\python.exe
C:\Users\yvnyl\AppData\Local\Microsoft\WindowsApps\python.exe


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('hw6_car_fuel_efficiency.csv')

In [5]:
df.head

<bound method NDFrame.head of       engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                     170            3.0       159.0     3413.433759   
1                     130            5.0        97.0     3149.664934   
2                     170            NaN        78.0     3079.038997   
3                     220            4.0         NaN     2542.392402   
4                     210            1.0       140.0     3460.870990   
...                   ...            ...         ...             ...   
9699                  140            5.0       164.0     2981.107371   
9700                  180            NaN       154.0     2439.525729   
9701                  220            2.0       138.0     2583.471318   
9702                  230            4.0       177.0     2905.527390   
9703                  270            3.0       140.0     2908.043477   

      acceleration  model_year  origin fuel_type         drivetrain  \
0             17.7        2003  Eu

In [6]:
df.columns = df.columns.str.lower()

In [9]:
df.dtypes

engine_displacement      int64
num_cylinders          float64
horsepower             float64
vehicle_weight         float64
acceleration           float64
model_year               int64
origin                  object
fuel_type               object
drivetrain              object
num_doors              float64
fuel_efficiency_mpg    float64
dtype: object

In [10]:
col_num = ['engine_displacement','num_cylinders','horsepower','vehicle_weight',
           'acceleration','model_year','num_doors']
col_cat = ['origin','fuel_type','drivetrain']

In [15]:
df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [19]:
df[col_num] = df[col_num].fillna(value=0)

In [20]:
df.isnull().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
[df_full_train,df_test] = train_test_split(df,test_size=0.2,random_state=1)
[df_train,df_val] = train_test_split(df_full_train,test_size=0.25,random_state=1)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(5822, 11)
(1941, 11)
(1941, 11)


In [29]:
y_test = df_test.fuel_efficiency_mpg.values
y_val = df_val.fuel_efficiency_mpg.values
y_train = df_train.fuel_efficiency_mpg.values

In [34]:
df_test = df_test.drop(columns=['fuel_efficiency_mpg'])
df_val = df_val.drop(columns=['fuel_efficiency_mpg'])
df_train = df_train.drop(columns=['fuel_efficiency_mpg'])

In [37]:
df_full_train = df_full_train.drop(columns=['fuel_efficiency_mpg'])

In [39]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
X_val = dv.transform(df_val.to_dict(orient='records'))
X_test = dv.transform(df_test.to_dict(orient='records'))
X_full_train = dv.transform(df_full_train.to_dict(orient='records'))

In [42]:
dv.get_feature_names_out()

array(['acceleration', 'drivetrain=All-wheel drive',
       'drivetrain=Front-wheel drive', 'engine_displacement',
       'fuel_type=Diesel', 'fuel_type=Gasoline', 'horsepower',
       'model_year', 'num_cylinders', 'num_doors', 'origin=Asia',
       'origin=Europe', 'origin=USA', 'vehicle_weight'], dtype=object)

In [36]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=1)

In [41]:
model.fit(X_train,y_train)

,criterion,'squared_error'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [45]:
from sklearn.tree import export_text

In [48]:
export_text(model,feature_names=dv.get_feature_names_out())

'|--- vehicle_weight <= 3022.11\n|   |--- value: [16.88]\n|--- vehicle_weight >  3022.11\n|   |--- value: [12.94]\n'

In [49]:
from sklearn.ensemble import RandomForestRegressor

In [50]:
model2 = RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=-1)

In [51]:
model2.fit(X_train,y_train)

,n_estimators,10
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [53]:
y_val_hat = model2.predict(X_val)

In [65]:
((y_val - y_val_hat)**2).mean()**0.5

np.float64(0.45957772230927263)